In [1]:
import pandas as pd
import numpy as np

# Load data

In [2]:
df = pd.read_csv('../dataset/HI-Small_Trans.csv')

#Rename columns
new_columns = [col.lower().replace(' ', '_') for col in df.columns]
df.columns = new_columns

In [2]:
df = pd.read_parquet('df.parquet2')

In [3]:
df.drop('index', axis=1, inplace=True)
df.head(5)

,timestamp,from_bank,to_bank,amount_received,amount_paid,is_laundering,year,month,day,hour,...,to_account,receiving_currency,payment_currency,payment_format,payment_payment,hop_2,hop_3,hop_4,hop_5,fan_in_degree
0,202209010008,1,1,14.26,14.26,0,2022,9,1,0,...,152883,0,0,4,-1,0,0,0,0,2
1,202209010008,1,1,14.26,14.26,0,2022,9,1,0,...,152883,0,0,4,-1,0,0,0,0,6
2,202209010008,1,1,14.26,14.26,0,2022,9,1,0,...,152883,0,0,4,-1,0,0,0,0,1
3,202209010008,1,1,14.26,14.26,0,2022,9,1,0,...,152883,0,0,4,-1,0,0,0,0,5
4,202209010008,1,1,14.26,14.26,0,2022,9,1,0,...,152883,0,0,4,-1,0,0,0,0,1


### Add possibly useful columns

In [4]:
df['same_account'] = df['from_account'] == df['to_account']
df['same_bank'] = df['from_bank'] == df['to_bank']
df['same_currency'] = df['receiving_currency'] == df['payment_currency']
df['same_amount'] = df['amount_received'] == df['amount_paid']

### Remove usesless columns

In [5]:
df.drop('timestamp', axis=1, inplace=True)
df.drop('from_bank', axis=1, inplace=True)
df.drop('to_bank', axis=1, inplace=True)
df.drop('from_account', axis=1, inplace=True)
df.drop('to_account', axis=1, inplace=True)
df.drop('year', axis=1, inplace=True)
df.drop('month', axis=1, inplace=True)

In [6]:
#Change position to is_laundering to last
column = df.pop("is_laundering")
df['is_laundering'] = column


df.head(5)

,amount_received,amount_paid,day,hour,minute,transaction_received_per_day,transaction_received_per_hour,transaction_received_per_minute,transaction_send_per_day,transaction_send_per_hour,...,hop_2,hop_3,hop_4,hop_5,fan_in_degree,same_account,same_bank,same_currency,same_amount,is_laundering
0,14.26,14.26,1,0,8,2,14,1,3,2,...,0,0,0,0,2,False,True,True,True,0
1,14.26,14.26,1,0,8,2,14,1,3,2,...,0,0,0,0,6,False,True,True,True,0
2,14.26,14.26,1,0,8,2,14,1,3,2,...,0,0,0,0,1,False,True,True,True,0
3,14.26,14.26,1,0,8,2,14,1,3,2,...,0,0,0,0,5,False,True,True,True,0
4,14.26,14.26,1,0,8,2,14,1,3,2,...,0,0,0,0,1,False,True,True,True,0


### Label encoding 

In [7]:
from sklearn.preprocessing import LabelEncoder

columns_to_encode = ['receiving_currency', 'payment_currency']
combined_values = df[columns_to_encode].values.ravel()
unique_combined_values = pd.unique(combined_values)
encoder = LabelEncoder()
encoder.fit(unique_combined_values)
for column in columns_to_encode:
    df[column] = encoder.transform(df[column])

encoder.fit(df['payment_format'])
df['payment_format'] = encoder.transform(df['payment_format'])

columns_to_convert = ['same_account', 'same_bank', 'same_currency', 'same_amount', 'is_laundering']
df[columns_to_convert] = df[columns_to_convert].astype(int)

df.head(5)

,amount_received,amount_paid,day,hour,minute,transaction_received_per_day,transaction_received_per_hour,transaction_received_per_minute,transaction_send_per_day,transaction_send_per_hour,...,hop_2,hop_3,hop_4,hop_5,fan_in_degree,same_account,same_bank,same_currency,same_amount,is_laundering
0,14.26,14.26,1,0,8,2,14,1,3,2,...,0,0,0,0,2,0,1,1,1,0
1,14.26,14.26,1,0,8,2,14,1,3,2,...,0,0,0,0,6,0,1,1,1,0
2,14.26,14.26,1,0,8,2,14,1,3,2,...,0,0,0,0,1,0,1,1,1,0
3,14.26,14.26,1,0,8,2,14,1,3,2,...,0,0,0,0,5,0,1,1,1,0
4,14.26,14.26,1,0,8,2,14,1,3,2,...,0,0,0,0,1,0,1,1,1,0


In [ ]:
df.drop('amount_received', axis=1, inplace=True)
df.drop('amount_paid', axis=1, inplace=True)

### Save dataframe

In [ ]:
df.to_csv('./preprocessed_data/preprocessed_data_small.csv', index=False)